In [ ]:
import requests
import glob
import json
import datetime as dt
import numpy as np
import pandas as pd
import time

In [ ]:
requestURL="https://www.imonnit.com/json/SensorDataMessages"

with open("./APIKey_imonnit.json") as f:
    headers=json.load(f)

targetSensorIDs={
    505723: "CH1A",
    505717: "CH1B",
    505766: "CH2A",
    505751: "CH2B",
    505805: "CH3A",
    505745: "CH3B",
    505786: "CH4A",
    505797: "CH4B",
    505768: "CH5A",
    505801: "CH5B",
    505796: "CH6A",
    505798: "CH6B",
    505778: "CH7A",
    505789: "CH7B"
}

initial_date=dt.datetime(2020, 5, 1)
latest_date=dt.datetime(2021, 6, 26)

In [ ]:
for k, v in targetSensorIDs.items():
    """
     1. set parameters
        a. set sensor ID (Done)
        b. set from Date and to Date (API limits 7 days window)

    """
    print(k)
    from_date=initial_date
    end_date=from_date+dt.timedelta(days=7)

    l=[]
    while from_date!=latest_date:

        from_string="{}-{}-{}".format(from_date.year, from_date.month, from_date.day)
        end_string="{}-{}-{}".format(end_date.year, end_date.month, end_date.day)

        parameters={
            "sensorID": k,
            "fromDate": from_string,
            "toDate": end_string
        }

        r=requests.post(requestURL, data=parameters, headers=headers)

        data=json.loads(r.content)
        
        for item in data['Result']:
            timestamp=item['MessageDate'].split(")")[0].split("(")[1]
            value=item['DataValues']
            l.append((timestamp, value))

        from_date=min(end_date+dt.timedelta(days=1), latest_date)
        end_date=min(from_date+dt.timedelta(days=7), latest_date)

    np.save("./imonnit/{}.npy".format(v), l)

In [ ]:

for file_ in sorted(glob.glob("./imonnit/*.npy")):
    max_date, min_date=dt.datetime(1970, 1, 1), dt.datetime(2022, 1, 1)
    data=np.load(file_)
    for item in data:
        timestamp=dt.datetime.fromtimestamp(float(item[0])/1000.)
        if timestamp>max_date:
            max_date=timestamp
        if timestamp<min_date:
            min_date=timestamp
    print(file_, min_date, max_date)
    

In [ ]:
for parent_folder in sorted(glob.glob("./imonnit/*")):
    folder_name=parent_folder.split("/")[-1]
    print(parent_folder)
    csv_list=glob.glob("{}/*.csv".format(parent_folder))
    if len(csv_list)==0:
        continue
    dfs=[]
    for csv_file in csv_list:
        print(csv_file)
        df=pd.read_csv(csv_file)
        df_date=[dt.datetime.strptime(item, "%m/%d/%Y %I:%M %p") for item in df['Date']]
        df_date=[[item.timestamp()*1000] for item in df_date]
        if 'Air Quality' in csv_file:
            df_data=[item.split("|")[:3] for item in df['Raw Data']]
        elif 'Aircon' in csv_file:
            df_data=[item.split(",")[:2] for item in df['Raw Data']]
        else:
            df_data=[[0] if item=="False" else [1] for item in df['Raw Data']]

        assert len(df_data)==len(df_date), "ERROR"

        df_file=[df_date[i]+df_data[i] for i in range(len(df_date))]
        dfs+=df_file
    dfs=np.array(dfs)

    for i in range(1, dfs.shape[1]):
        print(set(dfs[:,i]))
    np.save("./imonnit/{}.npy".format(folder_name), dfs)
    

In [ ]:
""